<a href="https://colab.research.google.com/github/ChandanNaik999/Biomedical-Text-Summarization/blob/main/Biomedical_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><b>Biomedical Text Summarization using K-means and Apriori Method</b></h1>

Presentation [here](https://docs.google.com/presentation/d/1YasEWkFZ_rdpzBRvciN-6wF-H_BT392uX7-et5cD1I4/edit?usp=sharing).

To run the code: Runtime->Run All
(The entire code takes around 30 min to run)

Note: Double click the section to open it


<h2><b>Setup</b></h2>

In [ ]:
#@title Time the code
import time
last_time = time.time()

Download the metamap from [here](https://metamap.nlm.nih.gov/MainDownload.shtml). After downloading the zip file upload it to google drive and paste the id in the variable "file_id" below 

In [ ]:
#@title Get Metamap binary from Google Drive
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://drive.google.com/uc?id="+id+"&export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

if __name__ == "__main__":
    file_id = '1DadjKQVk8v9ukKnund3O1ThMlEASc2HE'
    destination = '/content/public_mm_linux_main_2016.tar.bz2'
    download_file_from_google_drive(file_id, destination)

In [ ]:
#@title Metamap Unzip and Setup
%%shell

#cp /content/gdrive/My\ Drive/public_mm_linux_main_2016.tar.bz2 /content/
tar -xf public_mm_linux_main_2016.tar.bz2
export JAVA_HOME=/usr/
export PATH=$PATH:/content/public_mm/bin

In [ ]:
cd public_mm/

/content/public_mm


In [ ]:
#@title Metamap Installation
%%shell

#! /bin/bash
# install.sh - Public MetaMap Install Program
#
# public_mm/bin/install.sh, Wed Jul 16 10:21:21 2008, edit by Will Rogers
#
# This script reads environment variables BASEDIR and JAVA_HOME from
# parent process.
# 
if [ -x ./bin/date ]; then
  echo "$0 started `./bin/date`"  > ./install.log
else
  echo "$0 started `date`"  > ./install.log
fi
if [ -z $OS ]; then
    if [ -x ./bin/uname ]; then
	OS=`./bin/uname`
    else
	OS=`uname`
    fi
fi
case $OS in
    "Linux")
        # GNU libc version test
	REQ_MAJOR_VERSION=2
	REQ_MINOR_VERSION=12
	echo "glib minimun required version: $REQ_MAJOR_VERSION.$REQ_MINOR_VERSION"   >> ./install.log
	echo "glib minimun required version: $REQ_MAJOR_VERSION.$REQ_MINOR_VERSION" 
	;;
    "Windows_NT" | "MINGW32_NT-5.1" | "CYGWIN_NT-6.1")
	PATH=$PATH:./bin
	;;
esac

# load installrc if present.
if [ -r .installrc ]; then
    source .installrc
fi
BASEDIR=$PWD
BINDIR=$BASEDIR/bin

if [ "$JAVA_HOME" = "" ]; then
  javaprog=`which java`
  if [ $? -eq 0 ]; then
    javabindir=`dirname $javaprog`
    RC_JAVA_HOME=`dirname $javabindir`
  else
    RC_JAVA_HOME=""
  fi
else
  RC_JAVA_HOME=$JAVA_HOME
fi
JAVA_HOME=$RC_JAVA_HOME #added line
# echo  "Enter home path of JRE (JDK) [$RC_JAVA_HOME]: " 
# read JAVA_HOME
# if [ "$JAVA_HOME" = "" ]; then
#    JAVA_HOME=$RC_JAVA_HOME
# fi     

echo Using $JAVA_HOME for JAVA_HOME.
echo Using $JAVA_HOME for JAVA_HOME. >> ./install.log
echo ""

# setup WSD main and logging configuration files
if [ $OS = "MINGW32_NT-5.1" ]; then
    sed -e "s:@@basedir@@:\.\.:g" $BASEDIR/WSD_Server/config/disambServer.cfg.in > $BASEDIR/WSD_Server/config/disambServer.cfg
else 
    sed -e "s:@@basedir@@:$BASEDIR:g" $BASEDIR/WSD_Server/config/disambServer.cfg.in > $BASEDIR/WSD_Server/config/disambServer.cfg
fi
if [ -r $BASEDIR/WSD_Server/config/disambServer.cfg ]; then
  echo $BASEDIR/WSD_Server/config/disambServer.cfg generated
fi
if [ $OS = "MINGW32_NT-5.1" ]; then
sed -e "s:@@basedir@@:\.\.:g" $BASEDIR/WSD_Server/config/log4j.properties.in > $BASEDIR/WSD_Server/config/log4j.properties
else 
sed -e "s:@@basedir@@:$BASEDIR:g" $BASEDIR/WSD_Server/config/log4j.properties.in > $BASEDIR/WSD_Server/config/log4j.properties
fi
if [ -r $BASEDIR/WSD_Server/config/log4j.properties ]; then
  echo $BASEDIR/WSD_Server/config/log4j.properties generated
fi

if [ ! -d $BASEDIR/WSD_Server/log ]; then
  mkdir $BASEDIR/WSD_Server/log
  if [ -d $BASEDIR/WSD_Server/log ]; then
    echo Created directory $BASEDIR/WSD_Server/log
    echo Created directory $BASEDIR/WSD_Server/log >> ./install.log
  fi
fi

if [ ! -d $BASEDIR/MedPost-SKR/Tagger_server/log ]; then
  mkdir $BASEDIR/MedPost-SKR/Tagger_server/log
  if [ -d $BASEDIR/MedPost-SKR/Tagger_server/log ]; then
    echo Created directory $BASEDIR/Tagger_server/log
    echo Created directory $BASEDIR/Tagger_server/log >> ./install.log
  fi
fi

wsd_method_value="['AEC_METHOD']"

echo "Setting up bin directory scripts:"
echo "Setting up bin directory scripts:" >> ./install.log
# Setup all scripts generated from templates
for binscriptbase in $BASEDIR/bin/*.in
do
  binscript=`basename $binscriptbase .in`
  binscripttmp1=${binscript}.tmp1
  binscripttmp0=${binscript}.tmp0
  sed -e "s:@@basedir@@:$BASEDIR:g" $binscriptbase > $BINDIR/$binscripttmp1
  sed -e "s:@@java_home@@:$JAVA_HOME:g" $BINDIR/$binscripttmp1 > $BINDIR/$binscripttmp0
  sed -e "s:@@wsd_methods@@:$wsd_method_value:g" $BINDIR/$binscripttmp0 > $BINDIR/$binscript
  chmod +x $BINDIR/$binscript
  if [ -x $BINDIR/$binscript ]; then
    rm $BINDIR/$binscripttmp0
    rm $BINDIR/$binscripttmp1
    echo $BINDIR/$binscript generated.
    echo $BINDIR/$binscript generated. >> ./install.log
  fi
done

echo "Setting up test suite:"
echo "Setting up test suite:" >> ./install.log 2>&1

# Setup test script (runTest_<year>.sh runTest_<year>.sh)
for rtscripttmp in $BASEDIR/TestSuite/runTest_*.in
do
  rtscript=`basename $rtscripttmp .in`

  sed -e "s:@@basedir@@:$BASEDIR:g" $rtscripttmp > $BASEDIR/TestSuite/$rtscript
  chmod +x $BASEDIR/TestSuite/$rtscript
  if [ -x $BASEDIR/TestSuite/$rtscript ]; then
    echo $BASEDIR/TestSuite/$rtscript generated.
    echo $BASEDIR/TestSuite/$rtscript generated. >> ./install.log
  fi
done

# check for presence of lexicon, databases, tagger, and optional WSD.
echo "Checking for required datafiles"
echo "Checking for required datafiles" >> ./install.log
MISSINGFILES=0


checkforfile ()
{
    FLIST=`find $CHKDIR -name $CHKFILE -print | wc -l`
    if [ $FLIST == 0 ]; then
	echo "Warning: the $CHKFILE file is missing in $CHKDIR, cannot ensure correct operation of MetaMap without file!"
	echo "Warning: the $CHKFILE file is missing in $CHKDIR, cannot ensure correct operation of MetaMap without file!" >> ./install.log
	echo ""
	MISSINGFILES=`expr $MISSINGFILES + 1`
    fi
}

cd $BASEDIR

# DB Lexicon files
case $OS in
    "Windows_NT" | "MINGW32_NT-5.1" | "CYGWIN_NT-6.1")
	CHKDIR=./DB/\*.strict
	;;
    *)
	CHKDIR=./lexicon/data
	;;
esac
CHKFILE=dm_vars
checkforfile
CHKFILE=im_vars
checkforfile
CHKFILE=lex_form
checkforfile
CHKFILE=lex_rec
checkforfile
CHKFILE=norm_prefix
checkforfile

# MetaMap DB files
CHKDIR=./DB
CHKFILE=DB.\*.base
checkforfile
CHKFILE=DB.\*.[smr]\* 
checkforfile


# MedPost SKR tagger directory
TAGDIR=`find . -name MedPost-SKR -print | wc -l`
if [ $TAGDIR -eq 0 ]; then
  echo "Warning: Tagger directory is missing, cannot ensure correct operation of MetaMap without it!"
  echo "Warning: Tagger directory is missing, cannot ensure correct operation of MetaMap without it!" >> ./install.log
  echo ""
  MISSINGFILES=`expr $MISSINGFILES + 1`
else
  # MedPost SKR tagger files
  TAGFILES=`find ./MedPost-SKR/data -name \*.cur -print | wc -l`
  if [ $TAGFILES -eq 0 ]; then
    echo "Warning: Tagger index files are missing, cannot ensure correct operation of MetaMap without them!"
    echo "Warning: Tagger index files are missing, cannot ensure correct operation of MetaMap without them!" >> ./install.log
    echo ""
    MISSINGFILES=`expr $MISSINGFILES + 1`
  fi
fi

echo "Checking for optional datafiles (WSD)"
echo "Checking for optional datafiles (WSD)" >> ./install.log 2>&1 2>&1
MISSINGOPTIONS=0

# WSD Server directory
WSDDIR=`find . -name WSD_Server -print | wc -l`
if [ $WSDDIR -eq 0 ]; then
  echo "Warning: WSD Server directory is missing, MetaMap will not have WSD support without it!"
  echo "Warning: WSD Server directory is missing, MetaMap will not have WSD support without it!" >> ./install.log
  echo ""
  MISSINGOPTIONS=`expr $MISSINGOPTIONS + 1`
else 
  # WSD Server files
  # WSDFILES=`find ./WSD_Server -name wstv\* -print | wc -l`
  # if [ $WSDFILES == 0 ]; then
  #   echo "Warning: WSD Server index files are missing, MetaMap will not have WSD support without it!"
  #   echo "Warning: WSD Server index files are missing, MetaMap will not have WSD support without it!" >> ./install.log
  #   echo ""
  #   MISSINGOPTIONS=`expr $MISSINGOPTIONS + 1`
  # fi
  echo ""
fi

# echo BASEDIR=$BASEDIR > .installrc
# echo JAVA_HOME=$JAVA_HOME >> .installrc
echo "MISSINGOPTIONS=$MISSINGOPTIONS" >> ./install.log
if [ $MISSINGOPTIONS -ge 1 ]; then
  echo "!! WARNING: Some optional datafiles are missing, see install.log for more information. !!"
  echo "!! WARNING: Some optional datafiles are missing, see install.log for more information. !!" >> ./install.log
fi

echo "MISSINGFILES=$MISSINGFILES" >> ./install.log
if [ $MISSINGFILES -ge 1 ]; then
  echo "!! WARNING: Some necessary datafiles are missing, see install.log for more information. !!"
  echo "!! WARNING: Some necessary datafiles are missing, see install.log for more information. !!" >> ./install.log
else
  echo Public MetaMap Install complete.
  echo Public MetaMap Install complete. >> ./install.log
fi
echo "$0 ended `date`"  >> ./install.log

export BASEDIR

if [ -f $BASEDIR/bin/install_dfb.sh ]; then 
  echo  "Would like to use a custom data set with MetaMap (use data file builder)? [yN]:"
  read RESPONSE
  if [ "$RESPONSE" = "y" ]; then
     echo "running Data File Builder Install..."
     . $BASEDIR/bin/install_dfb.sh
  fi
fi

if [ -f $BASEDIR/bin/install_src.sh ]; then 
    echo "Running MetaMap source development environment setup..."
    . $BASEDIR/bin/install_src.sh
fi

if [ -f $BASEDIR/bin/install_javaapi.sh ]; then 
    echo "Running MetaMap Java API development environment setup..."
    . $BASEDIR/bin/install_javaapi.sh
fi

if [ -f $BASEDIR/bin/install_uima.sh ]; then 
    echo "Running MetaMap UIMA API development environment setup..."
    . $BASEDIR/bin/install_uima.sh
fi

echo ""
echo "Public MetaMap Install Settings:"
echo ""
echo "Public MetaMap basedir: $BASEDIR"
echo "Public MetaMap Program Dir: $BINDIR"
echo "Java Home dir: $JAVA_HOME"

echo "Public MetaMap basedir: $BASEDIR" >> ./install.log
echo "Public MetaMap Program Dir: $BINDIR" >> ./install.log
echo "Java Home dir: $JAVA_HOME" >> ./install.log
echo ""

exit 0

glib minimun required version: 2.12
Using /usr for JAVA_HOME.

/content/public_mm/WSD_Server/config/disambServer.cfg generated
/content/public_mm/WSD_Server/config/log4j.properties generated
Created directory /content/public_mm/WSD_Server/log
Created directory /content/public_mm/Tagger_server/log
Setting up bin directory scripts:
/content/public_mm/bin/metamap16 generated.
/content/public_mm/bin/metamap2016.TEMPLATE generated.
/content/public_mm/bin/metamap generated.
/content/public_mm/bin/skrmedpostctl generated.
/content/public_mm/bin/SKRrun.16 generated.
/content/public_mm/bin/uninstall.sh generated.
/content/public_mm/bin/wsdserverctl generated.
Setting up test suite:
/content/public_mm/TestSuite/runTest_2009.sh generated.
/content/public_mm/TestSuite/runTest_2010.sh generated.
/content/public_mm/TestSuite/runTest_2011.bat generated.
/content/public_mm/TestSuite/runTest_2011.sh generated.
/content/public_mm/TestSuite/runTest_2012.sh generated.
Checking for required datafiles
Check

In [ ]:
!git clone https://github.com/ChandanNaik999/pymetamap.git

Cloning into 'pymetamap'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 204 (delta 13), reused 13 (delta 7), pack-reused 179
Receiving objects: 100% (204/204), 45.38 KiB | 2.06 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [ ]:
cd pymetamap

/content/public_mm/pymetamap


In [ ]:
!python setup.py install --user

running install
running bdist_egg
running egg_info
creating pymetamap.egg-info
writing pymetamap.egg-info/PKG-INFO
writing dependency_links to pymetamap.egg-info/dependency_links.txt
writing top-level names to pymetamap.egg-info/top_level.txt
writing manifest file 'pymetamap.egg-info/SOURCES.txt'
writing manifest file 'pymetamap.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/pymetamap
copying pymetamap/__init__.py -> build/lib/pymetamap
copying pymetamap/ConceptLite.py -> build/lib/pymetamap
copying pymetamap/SubprocessBackend.py -> build/lib/pymetamap
copying pymetamap/SubprocessBackendLite.py -> build/lib/pymetamap
copying pymetamap/Concept.py -> build/lib/pymetamap
copying pymetamap/MetaMapLite.py -> build/lib/pymetamap
copying pymetamap/MetaMap.py -> build/lib/pymetamap
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.l

In [ ]:
from pymetamap import MetaMap

**Starting the SKR/Medpost Part-of-Speech Tagger Server:**

In [ ]:
!/content/public_mm/bin/skrmedpostctl start

$Starting skrmedpostctl: 
started.


**You can determine if the server are running by the command:**

In [ ]:
!ps -ef | grep java

root         234       1  0 12:23 ?        00:00:00 java -Dtaggerserver.port=1795 -DlexFile=/content/public_mm/MedPost-SKR/data/lexDB.serial -DngramOne=/content/public_mm/MedPost-SKR/data/ngramOne.serial -cp /content/public_mm/MedPost-SKR/Tagger_server/lib/taggerServer.jar:/content/public_mm/MedPost-SKR/Tagger_server/lib/mps.jar taggerServer
root         235      56  0 12:23 ?        00:00:00 /bin/bash -c ps -ef | grep java
root         237     235  0 12:23 ?        00:00:00 grep java


In [ ]:
#@title UMLS Concept mapping using Pymetamap
# metamap test script
# to run this script ensure that the "SKR/Medpost Part-of-Speech Tagger Server" has started
from pymetamap import MetaMap
mm = MetaMap.get_instance('/content/public_mm/bin/metamap16')

# sents = ['Heart Attack', 'John had a huge heart attack']
sents = ['developing world 15 cancers due infections helicobacter pylori hepatitis b ','hepatitis c human papillomavirus ','infection epstein–barr virus human immunodeficiency virus hiv']
concepts,error = mm.extract_concepts(sents,[1,2,3])
for concept in concepts:
  print(concept)

ConceptMMI(index='1', mm='MMI', score='32.62', preferred_name='Hepatitis B', cui='C0019163', semtypes='[dsyn]', trigger='["HEPATITIS B"-tx-1-"hepatitis b"-noun-0,"Infectious Hepatitis b"-tx-1-"infections hepatitis b"-noun-0]', location='TX', pos_info='64/11;33/10,64/11', tree_codes='C02.256.430.400;C02.440.435;C06.552.380.705.437')
ConceptMMI(index='1', mm='MMI', score='16.08', preferred_name='Helicobacter pylori', cui='C0079488', semtypes='[bact]', trigger='["HELICOBACTER PYLORI"-tx-1-"helicobacter pylori"-noun-0]', location='TX', pos_info='44/19', tree_codes='B03.440.500.550;B03.660.150.280.550')
ConceptMMI(index='1', mm='MMI', score='6.60', preferred_name='Infection', cui='C3714514', semtypes='[patf]', trigger='["Infections"-tx-1-"infections"-noun-0]', location='TX', pos_info='33/10', tree_codes='C01.539')
ConceptMMI(index='1', mm='MMI', score='3.45', preferred_name='Cancer Genus', cui='C0998265', semtypes='[euka]', trigger='["Cancer"-tx-1-"cancers"-noun-0]', location='TX', pos_info



---



<h2><b>Actual Code Starts Here</b></h2>

In [ ]:
#@title  Install Pip packages
!pip install wikipedia-api
!pip install rouge

# time taken
print(time.perf_counter() - last_time)

  Created wheel for wikipedia-api: filename=Wikipedia_API-0.5.4-cp36-none-any.whl size=13462 sha256=b7bf2dc52321c3150596df4c5aea6fad58b74a5c070962c2a07aeb01e57db298
  Stored in directory: /root/.cache/pip/wheels/bf/40/42/ba1d497f3712281b659dd65b566fc868035c859239571a725a
Successfully built wikipedia-api
-1606478298.278973


In [ ]:
#@title Get Data
import wikipediaapi


def get_article(topic="apple"):
    # f = open('/content/content.txt', 'r')
    # content = f.read()
    # f.close()

    # f = open('/content/summary.txt', 'r')
    # summary = f.read()
    # f.close()

    # return content, summary
    

    # get data from wiki
    wiki = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
    try:
        p_wiki = wiki.page(topic)
    except:
        print("Page " + topic + " Exists: False")
        quit()

    return p_wiki.text, p_wiki.summary

In [ ]:
#@title Cosine
from rouge import Rouge
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# from formatting import gen_serie
# from tokenizer import textblob_tokenizer


def cosine(texts, ref):
    vec = TfidfVectorizer(tokenizer=textblob_tokenizer,
                          stop_words='english',
                          use_idf=True)
    matrix = vec.fit_transform(texts)

    cosine_similarities = cosine_similarity(matrix[0:1], matrix).flatten()

    nb_sentences_in_base_summary = len(ref.split('.'))

    cosine_similarities = list(cosine_similarities)
    cos_results = []
    for i in range(0, nb_sentences_in_base_summary):
        n = cosine_similarities.index(max(cosine_similarities))
        cos_results.append(texts[n])
        del cosine_similarities[n]

    res = ' '.join(cos_results)

    r = Rouge()
    rouge = r.get_scores(res, ref)

    return gen_serie('Cosine Similarity', rouge, res, version = 'rouge-l'), gen_serie('Cosine Similarity', rouge, res, version = 'rouge-1'), gen_serie('Cosine Similarity', rouge, res, version = 'rouge-2')

In [ ]:
#@title TextRank Algorithm
import heapq
import re

# import nltk


def compute(sentences, stopwords, formatted):
    word_frequencies = {}
    sentence_scores = {}

    for word in nltk.word_tokenize(formatted):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    maximum_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word] / maximum_frequency)

    for sent in sentences:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]
    return sentence_scores


def text_rank(raw, text, ref):
    nb = len(ref.split('.'))

    formatted = re.sub('[^a-zA-Z]', ' ', raw)
    formatted = re.sub(r'\s+', ' ', formatted)

    scores = compute(text, nltk.corpus.stopwords.words('english'), formatted)
    sentences = heapq.nlargest(nb, scores, key=scores.get)

    return ' '.join(sentences)

In [ ]:
#@title Split and Tokenize sentences
import string

# import nltk
from textblob import TextBlob

import nltk
nltk.download('punkt')
nltk.download('stopwords')

def tokenize(raw):
    #3.1.2-3.1.3
    # tokenize the para to sentences
    sent = nltk.sent_tokenize(raw)
    return sent

def remove_punctuation_to_lower_case(sent):
    # remove punctuation marks and convert to lower case
    return list(map(lambda x: x.translate(dict.fromkeys(list(map(lambda x:ord(x),list(string.punctuation))), "")).lower(), sent))


def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#@title Cosine Similarity { form-width: "200px" }
from math import sqrt


def dot_product(v1, v2):
    """Get the dot product of the two vectors.

    if A = [a1, a2, a3] && B = [b1, b2, b3]; then
    dot_product(A, B) == (a1 * b1) + (a2 * b2) + (a3 * b3)
    true

    Input vectors must be the same length.

    """
    return sum(a * b for a, b in zip(v1, v2))


def magnitude(vector):
    """Returns the numerical length / magnitude of the vector."""
    return sqrt(dot_product(vector, vector))


def similarity(v1, v2):
    """Ratio of the dot product & the product of the magnitudes of vectors."""
    return dot_product(v1, v2) / (magnitude(v1) * magnitude(v2) + .00000000001)

In [ ]:
#@title Kmeans with Cosine Similarity { form-width: "200px" }
import random
class CosineKMeans(object):
    """K-Means clustering. Uses cosine similarity as the distance function."""

    def __init__(self, k, vectors,iter):
        assert len(vectors) >= k
        random.seed(20)
        self.centers = random.sample(vectors, k)
        self.clusters = [[] for c in self.centers]
        self.vectors = vectors
        self.iter = iter

    def update_clusters(self):
        """Determine which cluster center each `self.vector` is closest to."""
        def closest_center_index(vector):
            """Get the index of the closest cluster center to `self.vector`."""
            similarity_to_vector = lambda center: similarity(center,vector)
            center = max(self.centers, key=similarity_to_vector)
            return self.centers.index(center)

        self.clusters = [[] for c in self.centers]
        for vector in self.vectors:
             index = closest_center_index(vector)
             self.clusters[index].append(vector)

    def update_centers(self):
        """Move `self.centers` to the centers of `self.clusters`.

        Return True if centers moved, else False.

        """
        new_centers = []
        for cluster in self.clusters:
            center = [average(ci) for ci in zip(*cluster)]
            new_centers.append(center)

        if new_centers == self.centers:
            return False

        self.centers = new_centers
        return True

    def main_loop(self):
        """Perform k-means clustering."""
        self.update_clusters()
        # i = 0
        # while i < self.iter:
        #     self.update_clusters()
        #     i += 1
        while self.update_centers():
            self.update_clusters()


def average(sequence):
    return sum(sequence) / len(sequence)

In [ ]:
#@title Apriori Algorithm { form-width: "200px" }
import sys
from itertools import chain, combinations
from collections import defaultdict

def subsets(arr):
    """ Returns non empty subsets of arr"""
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])


def returnItemsWithMinSupport(itemSet, transactionList, minSupport, freqSet):
        """calculates the support for items in the itemSet and returns a subset
       of the itemSet each of whose elements satisfies the minimum support"""
        _itemSet = set()
        localSet = defaultdict(int)

        for item in itemSet:
                for transaction in transactionList:
                        if item.issubset(transaction):
                                freqSet[item] += 1
                                localSet[item] += 1

        for item, count in localSet.items():
                support = float(count)/len(transactionList)

                if support >= minSupport:
                        _itemSet.add(item)

        return _itemSet


def joinSet(itemSet, length):
        """Join a set with itself and returns the n-element itemsets"""
        return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def getItemSetTransactionList(data_iterator):
    transactionList = list()
    itemSet = set()
    for record in data_iterator:
        # record = record.split()
        transaction = frozenset(record)
        transactionList.append(transaction)
        for item in transaction:
            itemSet.add(frozenset([item]))              # Generate 1-itemSets
    return itemSet, transactionList


def runApriori(data_iter, original_text, minSupport=0.15, minConfidence=0.6):
    """
    run the apriori algorithm. data_iter is a record iterator
    Return both:
     - items (tuple, support)
     - rules ((pretuple, posttuple), confidence)
    """
    itemSet, transactionList = getItemSetTransactionList(data_iter)

    freqSet = defaultdict(int)
    largeSet = dict()
    # Global dictionary which stores (key=n-itemSets,value=support)
    # which satisfy minSupport

    # assocRules = dict()
    # Dictionary which stores Association Rules

    oneCSet = returnItemsWithMinSupport(itemSet,
                                        transactionList,
                                        minSupport,
                                        freqSet)

    currentLSet = oneCSet
    k = 2
    while(currentLSet != set([])):
        largeSet[k-1] = currentLSet
        currentLSet = joinSet(currentLSet, k)
        currentCSet = returnItemsWithMinSupport(currentLSet,
                                                transactionList,
                                                minSupport,
                                                freqSet)
        currentLSet = currentCSet
        k = k + 1

    def getSupport(item):
            """local function which Returns the support of an item"""
            return float(freqSet[item])/len(transactionList)

    toRetItems = []
    for key, value in largeSet.items():
        toRetItems.extend([(tuple(item), getSupport(item))
                           for item in value])

    # toRetRules = []
    # for key, value in list(largeSet.items())[1:]:
    #     for item in value:
    #         _subsets = map(frozenset, [x for x in subsets(item)])
    #         for element in _subsets:
    #             remain = item.difference(element)
    #             if len(remain) > 0:
    #                 confidence = getSupport(item)/getSupport(element)
    #                 if confidence >= minConfidence:
    #                     toRetRules.append(((tuple(element), tuple(remain)),
    #                                        confidence))
    return toRetItems#, toRetRules

In [ ]:
#@title Sentences evalution and selection { form-width: "200px" }
def searchInItemSet(j, items):
  for item, support in items:
    if set(item) == set(j):
      return support, True
  return 0, False

# cluster_senteces - actual sentences with the order number
# sentences - concepts_list 
def getSummary(cluster_sentences, sentences, items, needed_length):
	# from itertools import combinations
  sentencesScore = []
  for sentence in sentences:
    words = sentence #sentence.split()
    sentenceScore = 0
    for item, support in items:
      if set(item).issubset(set(words)):
        sentenceScore += len(item)*support
    sentencesScore.append(sentenceScore)
    #     return support, True
		# for i in range(1, len(words)):
		# 	comb = combinations(words, i)
		# 	for j in list(comb):
		# 		support, found = searchInItemSet(j,items)
		# 		if found:
		# 			sentenceScore += len(j)*support
		# sentencesScore.append(sentenceScore)
  # neededLength = int(compression_ratio * len(sentences))

  # sort the sentences according to the score and select the top sentences according to the compression ratio
  selectedSentences = sorted(list(zip(cluster_sentences, sentencesScore)), key = lambda val: val[1], reverse = True)[:needed_length]

  # since the selected sentences are not in the correct sequence we need to sort the selectedSentences in the order of the enumaration
  selectedSentences.sort(key = lambda val: val[0][0])

  topKSentences = []
  for sntnce, _ in selectedSentences:
    topKSentences.append(sntnce)
  # return the final summary
  return topKSentences

In [ ]:
#@title Rouge Evaluation Setup { form-width: "200px" }
import pandas as pd

columns = ['name', 'f', 'p', 'r', 'text']


def gen_serie(name, rouge, text, version = 'rouge-l'):
    return pd.Series([
        name,
        rouge[0][version]['f'],
        rouge[0][version]['p'],
        rouge[0][version]['r'],
        text],
        index=columns)

In [ ]:
#@title Proposed Algorithm { form-width: "200px" }
import warnings
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import numpy as np
import pandas as pd
import scipy.sparse
from rouge import Rouge
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances_argmin_min
import re
# from formatting import gen_serie
# from tokenizer import textblob_tokenizer, tokenize
 
warnings.filterwarnings("ignore")
 
 
def k_mean_distance(data, cx, cy, i_centroid, cluster_labels):
    distances = [np.sqrt((x - cx) ** 2 + (y - cy) ** 2) for (x, y) in data[cluster_labels == i_centroid]]
    return distances
 
def remove_url(message):
    return re.sub(r"http\S+", "", message)
 
def remove_citation(message):
    return re.sub(r"\[([0-9])*(–)?([0-9])*\]", "", message)
 
def remove_small_len(texts):
    arr = []
    for text in texts:
        if len(text.split()) >= 4:
            arr.append(text)
    return arr
 
def remove_stopwords(sentences):
    filtered_sentences = []
    stop_words = set(stopwords.words('english')) 
    for sentence in sentences:
        word_tokens = word_tokenize(sentence) 
        filtered_sentence = [] 
        for w in word_tokens: 
            if w not in stop_words: 
                filtered_sentence.append(w)
        filtered_sentences.append(" ".join(filtered_sentence))
    return filtered_sentences
 
def lt(sent):
    l = len(sent)
    if l > 250:
        import math
        k = math.ceil(l/250)
        sent = sent[::k]
    return sent
 
def delete_row_lil(mat, i):
    if not isinstance(mat, scipy.sparse.lil_matrix):
        raise ValueError("works only for LIL format -- use .tolil() first")
    mat.rows = np.delete(mat.rows, i)
    mat.data = np.delete(mat.data, i)
    mat._shape = (mat._shape[0] - 1, mat._shape[1])
    return mat
 
 
def cluster(matrix, clusters_nb):
    # create a map between the sentence vector and the sentence so that we can refer to sentence
    paragraph_map = dict(zip(map(str, matrix), [i for i in range(len(matrix))]))
    # print("matrix_len: ", len(matrix))
    #3.1.4
    # tokenize the sentences to words and remove stop words
    # vec = TfidfVectorizer(tokenizer=textblob_tokenizer,
    #                       stop_words='english',
    #                       use_idf=True)
    # matrix = vec.fit_transform(texts)
    # print(matrix)
 
    # Normal Method - Euclidean Distance
    # km = KMeans(n_clusters=clusters_nb, max_iter=10000, init='k-means++').fit(matrix)
    # centres are km.cluster_centers_
 
    # Cosine Similarity
    k_means = CosineKMeans(clusters_nb, matrix, 1000)
    print("Running Kmeans algorithm with cosine similarity")
    k_means.main_loop()
 
    def translator(clusters, paragraph_map):
        """Translate vectors back into paragraphs, to make them human-readable."""
        def item_translator(vector):
            return paragraph_map.get(str(vector))
 
        def cluster_translator(cluster):
            return list(map(item_translator, cluster))
 
        return list(map(cluster_translator, clusters))
 
    # here the paragraph map is the dict
    # key: string of vectors of the tokens
    # value: the sentence corresponding to that vector
    clusters = translator(k_means.clusters, paragraph_map)
 
    return clusters
 
 
def our_method(raw, ref):
    rougel = pd.DataFrame()
    rouge1 = pd.DataFrame()
    rouge2 = pd.DataFrame()
 
    # data preprocessing
 
    # remove url
    sents = remove_url(raw)
 
    # split text to sentence
    sents = tokenize(raw)
 
    # remove sentences of small len
    sents = remove_small_len(sents)
 
    # limit sentence
    sents = lt(sents)
    # sents = sents[::2]
 
    orig_sent = list(enumerate(sents))
    document_length = len(sents)
 
    print("Length of the entire document: ", document_length)
    
 
    # remove citations from text
    sents = list(map(remove_citation, sents))
 
    # remove punctuation and convert to lower case
    sents = remove_punctuation_to_lower_case(sents)
    
    # remove stop words
    sents = remove_stopwords(sents)
    print("Length of formatted sentences: ", len(sents))
    print("Sample formatted sentences: ", sents[:2])
 
    # UMLS Concept Mapping
    # Bag-of-words model: Each sentence has list of concepts associated with it
    print("Extracting biomedical concepts using Metamap")
    from pymetamap import MetaMap
    mm = MetaMap.get_instance('/content/public_mm/bin/metamap16')
 
    from collections import defaultdict, Counter
    concepts = defaultdict(list)
    all_concepts = [] # helper to calculate Sentence Frequency
    # turn on word sense disambiguation to remove ambiguity
 
    # ignored concepts - qualitative concept, quantitative concept, temporal concept, functional concept, idea or concept, intellectual product, mental process, spatial concept, and language
    # https://metamap.nlm.nih.gov/Docs/SemanticTypes_2018AB.txt
    ignrd_ctps = set(['qlco', 'qnco', 'tmco', 'ftcn', 'idcn', 'inpr', 'menp', 'spco', 'lang'])
 
    # why dint u do for entire para: metamap doesnt work when length of para is too big
    import ast 
    for idx, sentence in enumerate(sents):
        ctpts,error = mm.extract_concepts([sentence],[0], word_sense_disambiguation=False)
        for concept in ctpts:
            b = re.sub(r"([a-z]+)","\'\\1\'", concept.semtypes)
            res = ast.literal_eval(b) 
            if not set(res).intersection(ignrd_ctps): 
                # print(idx, concept.semtypes)
                concepts[idx].append(concept.preferred_name)
                all_concepts.append(concept.preferred_name)
            # IMPROVEMENT: concepts[concept.index].append((concept.preferred_name, concept.score)
 
    print("\n\nSample Concepts found: ")
    for i in range(3):
        print("\tSentence ", i, ": ", concepts[i])
 
 
    print("Biomedical Concepts found in ", len(concepts), " sentences")
 
    concepts_list = []
    for i in range(len(orig_sent)):
        if concepts[i]:
          concepts_list.append(concepts[i])
        else:
          concepts_list.append([])
 
    all_concepts_set = set(all_concepts)
    print("Total number of unique concepts found: ", len(all_concepts_set))
    all_concepts_dict = {k: v for v, k in enumerate(list(all_concepts_set))}
    matrix = [[0]*len(all_concepts_set) for _ in range(document_length)]
 
 
    print("Scoring the concepts")
    # Score( of each concept) = Concept Frequency * Sentence Frequency
    # Concept Frequency is the proportion of the number of occurrences of the 
    # concept Ci in the sentence Sj to the total number of concepts in the same sentence Sj :
    # Sentence frequency: Number of sentences having concept C / Total number of sentences
    total_number_of_sentences = document_length
    scored_concepts = defaultdict(dict)
    all_concepts = Counter(all_concepts)
    for i in concepts:
        total_number_concepts_in_sentence = len(concepts[i])
        counter = Counter(concepts[i])
        for j in concepts[i]:
            concept_frequency = counter[j]/total_number_concepts_in_sentence
            sentence_frequency = all_concepts[j]/total_number_of_sentences
            score = concept_frequency * sentence_frequency
            scored_concepts[i][j] = score
    # print("SCORED CONCEPTS", scored_concepts)
 
    # print(all_concepts_dict)
 
 
    for i in scored_concepts:
        for j in scored_concepts[i]:
              matrix[int(i)][all_concepts_dict[j]] = scored_concepts[i][j]
    # print(matrix)
 
    print("COMPRESSION RATIO  ======", len(tokenize(ref))/len(sents), "=======")
 
    # nb_clusters = 4
    for nb_clusters in range(3,8):
        print("".join(["-"]*50))
        print("Starting Clustering")
        clusters_with_index = cluster(matrix, nb_clusters)
        print("Clustering Done, number of clusters:: ", len(clusters_with_index))
        # print(clusters_with_index)
        # print(len(sents))
        # change cluster number to the cluster sentences
        clusters = []
        for i in clusters_with_index:
            temp = []
            for j in i:
                temp.append(concepts_list[j])
            clusters.append(temp)
        
        print("Starting Apriori for each cluster")
 
        f_res = ""
        for idx, cluster_ in enumerate(clusters):
            # items, rules = runApriori(cluster_, sents)
            items = runApriori(cluster_, sents)
            print("Apriori done for cluster ", idx)
 
            cluster_sentences = []
            for i in clusters_with_index[idx]:
                cluster_sentences.append([i, orig_sent[i][1]])
 
            res = getSummary(cluster_sentences,cluster_, items, int((len(tokenize(ref))/len(sents))*len(cluster_)))
            res.sort(key = lambda val:val[0])
 
            # print(res)
            
            for i, s in res:
                f_res += s  
        
        r = Rouge()
        
        # write summary to file
        f = open("/content/our_summary"+str(nb_clusters)+".txt","w")
        f.write(f_res)
        f.close()
 
        rouge = r.get_scores(f_res, ref)
        rougel = rougel.append(gen_serie('Our method('+str(nb_clusters)+') ', rouge, res,version = 'rouge-l'), ignore_index=True)
        rouge1 = rouge1.append(gen_serie('Our method('+str(nb_clusters)+') ', rouge, res,version = 'rouge-1'), ignore_index=True)
        rouge2 = rouge2.append(gen_serie('Our method('+str(nb_clusters)+') ', rouge, res,version = 'rouge-2'), ignore_index=True)
        
    return rougel, rouge1, rouge2

In [ ]:
#@title Evalution of All methods { form-width: "200px" }
import pandas as pd
from gensim.summarization import summarize
from rouge import Rouge
import sys
sys.setrecursionlimit(100000)

# from cosine_similarity import cosine
# from formatting import gen_serie, columns
# from kmean import kmean
# from scraper import get_article
# from text_rank import text_rank
# from tokenizer import tokenize
last_time = time.time()

def computeal(topic):
    raw, ref = get_article(topic)

    # tokenize the paras into sentences
    sent = tokenize(raw)
    sent = lt(sent)
    

    rougel = pd.DataFrame()
    rouge1 = pd.DataFrame()
    rouge2 = pd.DataFrame()

    ratio = len(ref) / len(raw)

    # KMean
    kmean_res = our_method(raw, ref)
    rougel = rougel.append(kmean_res[0])
    rouge1 = rouge1.append(kmean_res[1])
    rouge2 = rouge2.append(kmean_res[2])

    # TextRank
    result = text_rank(raw, sent, ref)
    r = Rouge()
    rouge = r.get_scores(result, ref)
    rougel = rougel.append(gen_serie('TextRank', rouge, result,version = 'rouge-l'), ignore_index=True)
    rouge1 = rouge1.append(gen_serie('TextRank', rouge, result,version = 'rouge-1'), ignore_index=True)
    rouge2 = rouge2.append(gen_serie('TextRank', rouge, result,version = 'rouge-2'), ignore_index=True)

    # Gensim
    ret = summarize(raw, ratio)
    r = Rouge()
    rouge = r.get_scores(ret, ref)
    rougel = rougel.append(gen_serie('Gensim', rouge, ret,version = 'rouge-l'), ignore_index=True)
    rouge1 = rouge1.append(gen_serie('Gensim', rouge, ret,version = 'rouge-1'), ignore_index=True)
    rouge2 = rouge2.append(gen_serie('Gensim', rouge, ret,version = 'rouge-2'), ignore_index=True)

    # Cosine
    res = cosine(sent, ref)
    rougel = rougel.append(res[0], ignore_index=True)
    rouge1 = rouge1.append(res[1], ignore_index=True)
    rouge2 = rouge2.append(res[2], ignore_index=True)

    
    print("========= Text summarization Complete =========")

    # Rearrange columns
    rougel = rougel[columns]
    del rougel['text']
    rougel.to_csv('/content/output_rougel.csv')
    print("\n========= ROUGE L =========")
    print(rougel)


    # Rearrange columns
    rouge1 = rouge1[columns]
    del rouge1['text']
    rouge1.to_csv('/content/output_rouge1.csv')
    print("\n========= ROUGE 1 =========")
    print(rouge1)

    # Rearrange columns
    rouge2 = rouge2[columns]
    del rouge2['text']
    rougel.to_csv('/content/output_rouge2.csv')
    print("\n========= ROUGE 2 =========")
    print(rouge2)


    # return df.to_json(orient='records'), df

computeal("heart")

print("".join(["-"]*50))
print("Total time taken: ", int((time.time()-last_time)/60), " minutes and ", (time.time()-last_time)%60," seconds")

Length of the entire document:  168
Length of formatted sentences:  168
Sample formatted sentences:  ['heart muscular organ animals pumps blood blood vessels circulatory system', 'commonly right atrium ventricle referred together right heart left counterparts left heart']
Extracting biomedical concepts using Metamap


Sample Concepts found: 
	Sentence  0 :  ['Blood Circulation', 'Myocardium', 'Animals', 'Muscle', 'Heart', 'Blood Vessel', 'Pump Device Component', 'pump (device)', 'Vascular System', 'animal allergen extracts', 'Entire heart', 'Organ']
	Sentence  1 :  ['Heart Ventricle', 'Heart Atrium', 'Entire right ventricle', 'Right ventricular structure', 'Ventricle', 'Entire right atrium', 'Right atrial structure', 'Right side of heart', 'Entire atrium']
	Sentence  2 :  ['Pericardial sac structure', 'Heart', 'Body Fluids', 'Contain (action)', 'Entire heart', 'Entire pericardium', 'Liquid substance', 'fluid - substance', 'ADCY10 gene', 'spindle assembly checkpoint', 'subapical complex